### Download the tutorial overlay

The `ps_gpio_kv260.bit` and `ps_gpio_kv260.hwh` files are in the `ps_gpio` directory local to this folder. 
The bitstream can be downloaded using the PYNQ `Overlay` class. 

In [ ]:
from pynq import Overlay
ps_gpio_design = Overlay("./ps_gpio_kv260.bit")

## PYNQ GPIO class

The PYNQ GPIO class will be used to access the PS GPIO. 

In [ ]:
from pynq import GPIO
from time import sleep
from pynq import Overlay
ps_gpio_design = Overlay("./ps_gpio_kv260.bit")

### GPIO Assignment

In [ ]:
# Define GPIO Inputs
emergency_button = GPIO(GPIO.get_gpio_pin(3), 'in')  # Emergency button
floor_req_b1 = GPIO(GPIO.get_gpio_pin(7), 'in')  # First bit of the floor request
floor_req_b2 = GPIO(GPIO.get_gpio_pin(6), 'in')  # Second bit of the floor request

# Define GPIO Outputs
motor_up_down = GPIO(GPIO.get_gpio_pin(2), 'out')  # Elevator Up/Down
stop_go = GPIO(GPIO.get_gpio_pin(1), 'out')  # Elevator Stop/Go
open_close = GPIO(GPIO.get_gpio_pin(5), 'out')  # Elevator Door Open/Close
ssd_b1 = GPIO(GPIO.get_gpio_pin(0), 'out')  # SSD bit 1
ssd_b2 = GPIO(GPIO.get_gpio_pin(4), 'out')  # SSD bit 2

### Variable Assignment

In [ ]:
# Initialize the system
ssd_b1.write(0)
ssd_b2.write(0)
current_floor = 1
floor_requests = [False] * 4
direction_up = True
state = "IDLE"

### Loop Code

In [ ]:
# Function to update the 7-segment display with the current floor number
def update_display(floor):
    binary = [(0, 0), (0, 1), (1, 0), (1, 1)]
    ssd_b1.write(binary[floor-1][0])
    ssd_b2.write(binary[floor-1][1])

# Function to read floor requests from the GPIO pins and return the requested floor number
def read_floor_requests():
    request = floor_req_b1.read() * 2 + floor_req_b2.read()
    return request + 1  # Floors are 1-indexed

# Function to move the elevator to the requested floor
def move_to_floor(target_floor):
    global current_floor, direction_up

    # Determine direction of travel
    direction_up = target_floor > current_floor
    motor_up_down.write(int(direction_up))
    
    # Start moving the elevator
    while current_floor != target_floor:
            if current_floor < target_floor:
                motor_up_down.write(1)  # Move up, light LED for up
                current_floor += 1
            else:
                motor_up_down.write(0)  # Move down, light LED for down
                current_floor -= 1

            stop_go.write(0)  # Start moving, turn off stop LED
            update_display(current_floor)  # Update display at each floor
            sleep(4)  # Simulate the time to move between floors

        # Ensure doors open/close only at the final target floor
            if current_floor == target_floor:
                state = "DOOR_OPEN"

            if current_floor == 4:
                direction_up = False
            elif current_floor == 1:
                direction_up = True


            # Check for emergency stop
            if emergency_button.read():
                break
    
    # Stop the elevator when the target floor is reached
    stop_go.write(1)
            
# Function to handle emergency situations            
def handle_emergency():
    global current_floor, direction_up, floor_requests, state
    # Clear all floor requests
    floor_requests = [False] * 4
    # If not on the first floor, move to the first floor
    if current_floor != 1:
        direction_up = False
        move_to_floor(1)  # Moving to floor 1 as part of emergency protocol
    # Open doors at the first floor
    open_doors()
    state = "IDLE"  # Reset state to IDLE after handling emergency

# Function to open the doors, wait for a while, and then close them
def open_doors():
    open_close.write(1)  # Open doors
    sleep(5)  # Wait for people to exit and enter
    open_close.write(0)  # Close doors

# Main control loop
while True:
    try:
        # Check and handle emergency button press
        if emergency_button.read():
            state = "EMERGENCY"
        
        # FSM logic based on the current state
        if state == "IDLE":
            # Check for floor requests and transition to MOVING state
            requested_floor = read_floor_requests()
            if requested_floor > 0:
                floor_requests[requested_floor - 1] = True
                state = "MOVING"
                target_floor = requested_floor

        elif state == "MOVING":
            # Move elevator to the target floor and transition to DOOR_OPEN state
            move_to_floor(target_floor)
            state = "DOOR_OPEN"

        elif state == "DOOR_OPEN":
            # Open doors and then transition back to IDLE state
            open_doors()
            state = "IDLE"

        elif state == "EMERGENCY":
            # Handle emergency by moving to floor 1 and then transition to IDLE
            handle_emergency()

        # A short delay to prevent high CPU utilization in the loop
        sleep(0.1)

    except Exception as e:
        print(f"Error: {e}")
        # Decide whether to break or continue based on the error handling policy
        break